### Loading Data

In [4]:
%reload_ext autoreload
%autoreload 2

In [7]:
import os, json, random, re
import pandas as pd
from tqdm import tqdm
import utils

In [9]:
data_f = utils.data_f
train_csv = pd.read_csv(data_f + 'train.csv')

print(train_csv.shape, 'train_csv shape')
train_csv.sample(10)

(19661, 5) train_csv shape


,Id,pub_title,dataset_title,dataset_label,cleaned_label
9233,8a47e982-d59d-4fb0-90b2-c867f7a3493b,APOE and BDNF polymorphisms moderate amyloid β...,Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
4105,94c3a80d-58f5-4802-b75d-5022be2ee98a,False positives in neuroimaging genetics using...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni
7529,bdbb6833-7a47-4bea-9f26-a179f07f69cc,Measurement Efficiency for Fixed-Precision Mul...,Trends in International Mathematics and Scienc...,Trends in International Mathematics and Scienc...,trends in international mathematics and scienc...
1619,01f22cf2-d19d-4eb0-b169-cb25b80c8f0a,The BRAINS Program: Transforming Career Develo...,Survey of Earned Doctorates,Survey of Earned Doctorates,survey of earned doctorates
4861,3b0f0f2e-f597-4b1d-9624-aea6f9f615fe,Automated Mapping of Hippocampal Atrophy in 1-...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni
10743,e6a79b4a-c771-48c7-a8be-60626d94eabe,Is tau in the absence of amyloid on the Alzhei...,Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
9352,4391c149-122a-408e-ba9b-36a9eaf4588e,3DSiameseNet to Analyze Brain MRI,Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
10695,0f480504-60c9-427a-96e9-a88180dcd481,"Associations between cognitive, functional, an...",Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
12337,5b53627e-7c4a-486d-a98e-1154aba452df,Teacher Salaries and Teacher Unions: A Spatial...,Common Core of Data,Common Core of Data,common core of data
14744,73709540-a811-45b9-b1ba-2db98e8a7161,Higher Education in Science and Engineering,Survey of Doctorate Recipients,Survey of Doctorate Recipients,survey of doctorate recipients


In [10]:
# documents
print(train_csv.Id.unique().shape, 'unique ids') # repeated IDs means documents have multiple dataset labels
most_repeated_ids = train_csv.Id.value_counts()[:10]
print(most_repeated_ids)

(14316,) unique ids
170113f9-399c-489e-ab53-2faf5c64c5bc    22
430aa11c-0283-411b-8edc-08f5df3db258    12
dd6df078-2010-42fa-ad30-9c302d2ba55b    12
0cbde738-c072-47e7-92b5-b6ccb5661f2d    11
c82e4146-9a3e-45bd-8bf8-4866140b7412    10
3d61cdd8-240e-4202-9837-db4263fd196a    10
b262c60b-5b5f-4765-93bd-fb88458b4d7f     9
c4e22d3d-a953-4525-9f4f-97af0132c276     9
5509b730-b7d5-459f-aa6b-72d84a81e97f     9
4a2bd823-e642-4ad9-b8e6-796ae4db9734     9
Name: Id, dtype: int64


In [11]:
## datasets
print(train_csv.cleaned_label.unique().shape, 'unique_cleaned labels (# of datasets)')
print(train_csv.cleaned_label.value_counts()[:10], 'most reffered datasets, with freq')
train_csv.cleaned_label.value_counts().describe() # most of the top dataset are mentioned over 10 times, while each dataset is mentioned at least twice

(130,) unique_cleaned labels (# of datasets)
adni                                                     3673
alzheimer s disease neuroimaging initiative adni         2400
trends in international mathematics and science study    1163
baltimore longitudinal study of aging                    1156
early childhood longitudinal study                       1011
education longitudinal study                              676
census of agriculture                                     643
agricultural resource management survey                   623
national education longitudinal study                     550
rural urban continuum codes                               490
Name: cleaned_label, dtype: int64 most reffered datasets, with freq


count     130.000000
mean      151.238462
std       425.998595
min         1.000000
25%         2.000000
50%        12.000000
75%       106.500000
max      3673.000000
Name: cleaned_label, dtype: float64

In [78]:
import prepare_data as prep

In [88]:
s = ['beginning postsecondary students longitudinal study', 'education longitudinal study', 'beginning postsecondary students']

prep.keep_longest_label(s)

['education longitudinal study',
 'beginning postsecondary students longitudinal study']

In [91]:
# ids = train_csv.Id.sample(10).unique()
labels_map = {i: train_csv[train_csv.Id==i]['cleaned_label'].values.tolist() for i in ids}

data = prep.DataPrep(labels_map)
tagged_data = data.prep()

processing...: 100%|██████████| 254/254 [00:00<00:00, 2870.43it/s]


In [105]:
d = data.meta['801d8b49-b0db-45e9-9a1f-3b54e22c154f_10']
text = d['clean_text']
labels = data.labels_map['801d8b49-b0db-45e9-9a1f-3b54e22c154f']
idx = prep.get_indx(text, labels)
tokens, tags = prep.split_and_tag(text, idx)
print([(i, j) for i, j in zip(tokens, tags) if j!='O'])


[('baltimore', 'B'), ('longitudinal', 'I'), ('study', 'I'), ('of', 'I'), ('aging', 'I'), ('blsa', 'I')]


In [106]:
idx

[(2843, 2886)]

In [104]:
text[2843: 2887]

'baltimore longitudinal study of aging blsa l'

In [97]:
for c, i in enumerate(tagged_data[:30]):
    tokens = i['tokens']    
    tags = i['tags']
    para_id = i['id']
    doc_id = para_id.split('_')[0]
    labels = data.labels_map[doc_id]
    text = data.meta[para_id]['clean_text']
    
    tagging = ' '.join([str((i, j)) for i, j in zip(tokens, tags) if j != 'O'])
    
    if tagging != []:
        print(para_id)
        print('lables=', labels, '\n',
            'tagging=', tagging)

04def0f2-de94-45be-ad6a-6b79e1571b59_3
lables= ['national education longitudinal study', 'education longitudinal study'] 
 tagging= ('national', 'B') ('education', 'I') ('longitudinal', 'I') ('study', 'I')
72d0eb13-7caa-4b8b-af9d-43b8a36f1189_15
lables= ['adni'] 
 tagging= ('adni', 'B')
72d0eb13-7caa-4b8b-af9d-43b8a36f1189_16
lables= ['adni'] 
 tagging= ('adni', 'B')
801d8b49-b0db-45e9-9a1f-3b54e22c154f_10
lables= ['baltimore longitudinal study of aging blsa ', 'baltimore longitudinal study of aging'] 
 tagging= ('baltimore', 'B') ('longitudinal', 'I') ('study', 'I') ('of', 'I') ('aging', 'I') ('blsa', 'I') ('l', 'I')
4f28e472-28ef-486a-881b-116b3713acf2_0
lables= ['adni'] 
 tagging= ('adnig', 'B')
4f28e472-28ef-486a-881b-116b3713acf2_1
lables= ['adni'] 
 tagging= ('adnig', 'B')
4f28e472-28ef-486a-881b-116b3713acf2_10
lables= ['adni'] 
 tagging= ('adni', 'B')
c28abc20-abea-4ecc-bac6-81d78c1692c2_69
lables= ['beginning postsecondary students longitudinal study', 'baccalaureate and beyon

In [131]:
# !pip install --upgrade transformers 

In [26]:
import transformers
transformers.__version__

task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


# a = tokenizer("Hello, this is one sentence!")



# s = sections_target[4]
# text_to_tokenize = s.text
# target_char_index = s.text.index(s.target)
# target_span = (target_char_index, target_char_index + len(s.target))

In [40]:
def get_dataset_dict(sections):
    tagged_data = [tag_section(s) for s in sections]
    return dict(
        mapping = dict(
            id=list(range(len(tagged_data))),
            tokens=[i for i,j in tagged_data],
            ner_tags=[list(map(ner_tags.get, j)) for i,j in tagged_data]            
        )
    )

from datasets import Dataset, load_metric

data = Dataset.from_dict(**get_dataset_dict(sections_target))


In [41]:
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(examples["tokens"], padding='max_length', truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [42]:
tokenize_and_align(data[:2])

{'input_ids': [[101, 2023, 2817, 2109, 2951, 2013, 1996, 2120, 2495, 20134, 2817, 11265, 4877, 6070, 2000, 11628, 1996, 3896, 1997, 7037, 10316, 3454, 2005, 2152, 2082, 2493, 2006, 2267, 3014, 18759, 3672, 1996, 2817, 2036, 2988, 3251, 1996, 14670, 1997, 7037, 10316, 3454, 2020, 2367, 2005, 2034, 4245, 2267, 2493, 6431, 2493, 3005, 3008, 2018, 3230, 2012, 2560, 2070, 2267, 1999, 2804, 1037, 27024, 4106, 4311, 2006, 1996, 4254, 1997, 2367, 8310, 1997, 7037, 10316, 2607, 2635, 1998, 2267, 3014, 18759, 3672, 7037, 10316, 3454, 3749, 2267, 2504, 4083, 6322, 2005, 2152, 2082, 2493, 1996, 3454, 3749, 2267, 5352, 1998, 2030, 1996, 4495, 2000, 7796, 2267, 6495, 2005, 2493, 2096, 2145, 1999, 2152, 2082, 1996, 8830, 2177, 1999, 1996, 2817, 2001, 11539, 1997, 11265, 4877, 6818, 2040, 3230, 1037, 8466, 8586, 29067, 2854, 2082, 1998, 2040, 4194, 1999, 1037, 7037, 10316, 2565, 2096, 1999, 2152, 2082, 1050, 26839, 1996, 2817, 3166, 2109, 17678, 6132, 3012, 3556, 9844, 4725, 2000, 3443, 1037, 7831, 21

In [43]:
tokenized_data = data.map(tokenize_and_align, batched=True)

In [44]:
label_list = list(ner_tags.keys())

In [45]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [53]:
args = TrainingArguments(
    f"test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [54]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [55]:
# !pip install seqeval
metric = load_metric("seqeval")

In [120]:
labels = data[f"{task}_tags"]
metric.compute(predictions=labels, references=labels)

{'_': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 725},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [49]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [56]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,No log,0.003569,0.812500,0.982558,0.889474,0.999039,618.199400,1.173000
2,No log,0.000921,0.955407,0.996512,0.975526,0.999769,617.634000,1.174000
3,0.017500,0.000455,0.986207,0.997674,0.991908,0.999921,617.676800,1.174000


TrainOutput(global_step=546, training_loss=0.016047373226410522, metrics={'train_runtime': 8186.2417, 'train_samples_per_second': 0.067, 'total_flos': 443425633459200.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 3398427, 'init_mem_cpu_peaked_delta': 89208, 'train_mem_cpu_alloc_delta': 474775, 'train_mem_cpu_peaked_delta': 94605776})

In [52]:
batch_size=4

In [62]:
!ls

bamnet_exp  coleridge_data  EmbedKGQA  mlruns  random.ipynb  runs  test-ner


In [63]:
test_csv = pd.read_csv(data_f + 'sample_submission.csv')
test_csv

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN


In [75]:
def prepare_datasets(csv):
    sections = [Section(jdata, csv_ind) for csv_ind in tqdm(csv.index) for jdata in get_json(from_csv(csv_ind, 'Id', csv))]
    data = Dataset.from_dict(**get_dataset_dict(sections_target))
    tokenized_data = data.map(tokenize_and_align, batched=True)
    return tokenized_data, sections   
    

In [76]:
test_tokenized, test_sections = prepare_datasets(test_csv)

100%|██████████| 4/4 [00:00<00:00, 33.74it/s]


In [104]:
test_tokenized

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'tokens'],
    num_rows: 118
})

In [106]:
test_tokenized.remove_columns(['labels', 'ner_tags'])

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'tokens'],
    num_rows: 118
})

In [ ]:
model()

In [108]:
trainer.compute_metrics = None

In [115]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# results = metric.compute(predictions=true_predictions, references=true_labels)
# results

In [110]:
# predictions = numpy.array([[label_list[p] for p in pred] for pred in predictions])

In [165]:
def decode_labels(data, lables, skip='[SEP]'):
    
    for ix in tqdm(range(len(data))):
        # ix = 4
        predicted = [(i, j) for i, j in enumerate(labels[ix]) if j in [1, 2]]
        s = [(data[ix]['input_ids'][i], j) for i, j in predicted]
        s = tokenizer.decode([i for i, j in s])
        if s not in skip: print(ix, s)

In [166]:
len(test_tokenized)

118

In [167]:
# a page of requirement of KG paper

In [168]:
random_ids = random.choices(range(0, 700), k=5)
data = [{'input_ids': tokenized_data['input_ids'][i]} for i in random_ids]
labels = [tokenized_data['labels'][i] for i in random_ids]

decode_labels(data, labels)

100%|██████████| 5/5 [00:00<00:00, 1007.28it/s]

0 slosh model
1 slosh model
2 noaa tidal station
3 alzheimers disease neuroimaging initiative
4 national education longitudinal study


3

In [155]:
random_ids

[153, 353, 18, 139, 454]

In [169]:
train_csv.iloc[random_ids]

,Id,pub_title,dataset_title,dataset_label,cleaned_label
564,51505a9b-b5ce-4132-a5fb-ec6aaf69c1b5,Kidney regeneration in fish.,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging blsa
510,149cd856-d8b4-4783-8df4-26cfd6287eeb,Space geodetic determination of spatial variab...,NOAA Tide Gauge,NOAA National Water Level Observation Network,noaa national water level observation network
375,acedff21-5ea9-4cf0-b207-a421698e03ec,Effects of Control Power and Inceptor Sensitiv...,"Sea, Lake, and Overland Surges from Hurricanes",SLOSH model,slosh model
681,76f6b7f5-2b56-4410-87f8-dbc2f5f8a7e7,Prevalence and Outcomes of Frailty in Korean E...,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging blsa
264,6fd43867-841f-4490-b3d1-f38eb60b4f08,A General Panel Model with Random and Fixed Ef...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [171]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForTokenClassification

In [172]:
trainer.save_model??

Signature: trainer.save_model(output_dir:Union[str, NoneType]=None)
Source:   
    def save_model(self, output_dir: Optional[str] = None):
        """
        Will save the model, so you can reload it using :obj:`from_pretrained()`.

        Will only save from the main process.
        """
        if is_torch_tpu_available():
            self._save_tpu(output_dir)
        elif (
            ShardedDDPOption.ZERO_DP_2 in self.args.sharded_ddp or ShardedDDPOption.ZERO_DP_3 in self.args.sharded_ddp
        ):
            state_dict = self.model.state_dict()
            if self.is_world_process_zero():
                self._save(output_dir, state_dict=state_dict)
        elif self.is_world_process_zero():
            self._save(output_dir)
File:      ~/covid-env/lib/python3.6/site-packages/transformers/trainer.py
Type:      method


Init signature:
Trainer(
    model:torch.nn.modules.module.Module=None,
    args:transformers.training_args.TrainingArguments=None,
    data_collator:Union[DataCollator, NoneType]=None,
    train_dataset:Union[torch.utils.data.dataset.Dataset, NoneType]=None,
    eval_dataset:Union[torch.utils.data.dataset.Dataset, NoneType]=None,
    tokenizer:Union[_ForwardRef('PreTrainedTokenizerBase'), NoneType]=None,
    model_init:Callable[[], transformers.modeling_utils.PreTrainedModel]=None,
    compute_metrics:Union[Callable[[transformers.trainer_utils.EvalPrediction], Dict], NoneType]=None,
    callbacks:Union[List[transformers.trainer_callback.TrainerCallback], NoneType]=None,
    optimizers:Tuple[torch.optim.optimizer.Optimizer, torch.optim.lr_scheduler.LambdaLR]=(None, None),
)
Source:        
class Trainer:
    """
    Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.

    Args:
        model (:class:`~transformers.PreTrainedModel` 

In [176]:
# !mkdir -p saved_models/coleridge/distill-bert-finetuned1
trainer.save_model('saved_models/coleridge/distill-bert-finetuned1')

In [177]:
!du -sh saved_models/

254M	saved_models/


In [178]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13329176     7779792      377920       25580     5171464     5148136
Swap:             0           0           0


In [2]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        4.2G        3.5G         24M        4.9G        8.1G
Swap:            0B          0B          0B
